In [125]:
!pip install -qU pandas bs4
!pip install -qU langchain langchain-community langchain-chroma
!pip install -qU langchain-google-genai langchain-cohere
!pip install -qU langchain-groq
!pip install -qU pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.8 MB/s eta 0:00:00


In [ ]:
from langchain_cohere import CohereEmbeddings, ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from google.colab import userdata

import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [109]:
# API
COHERE_API = userdata.get('COHERE_API')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')


def groq_llm():
  llm = ChatGroq(model="llama3-8b-8192", groq_api_key=GROQ_API_KEY)
  return llm

def co_llm():
  llm = ChatCohere(cohere_api_key=COHERE_API)
  return llm

def cohere_embed():
  embeddings = CohereEmbeddings(model = "embed-multilingual-v3.0", cohere_api_key=COHERE_API)
  return embeddings


embeddings = cohere_embed()
#vector = embeddings.embed_query("hello")
#print(vector[:5])
#print(len(vector))

llm = groq_llm()
res = llm.invoke("Write a ballad about Ethereum")
print(res.content)

(Verse 1)
In the dawn of blockchain's prime
A vision took shape, a dream divine
Ethereum rose, a beacon bright
A platform born, to shine with all its might

(Chorus)
Oh, Ethereum, Ethereum, a world of code
Where smart contracts thrive, and innovation abides
With gas and transactions, it's a world to explore
Ethereum, the land of decentralized lore

(Verse 2)
Gas prices rose, and fees took flight
But Vitalik's vision, remained in sight
The blockchain's heart, a beating strong
As decentralized apps, began to throng

(Chorus)
Oh, Ethereum, Ethereum, a world of code
Where smart contracts thrive, and innovation abides
With gas and transactions, it's a world to explore
Ethereum, the land of decentralized lore

(Bridge)
The DAO's fate, a lesson learned
But Vitalik's vision, remained unfurled
The community rallied, through trials and strife
And Ethereum's spirit, remained alive

(Verse 3)
Solidity's language, a new syntax born
As developers flocked, to Ethereum's form
Decentralized finance, a 

In [128]:
# get pdf
from langchain_community.document_loaders import PyPDFLoader

file_path = ("./WEF_Global_Risks_Report_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

len(pages)

143

In [111]:
# get weburl text
def get_weburl_text(url):
  loader = WebBaseLoader(
      web_paths=(url,),
      bs_kwargs=dict(
          parse_only=bs4.SoupStrainer(
              class_=("post-content", "post-title", "post-header")
          )
      ),
  )
  text = loader.load()
  return text

URL = "https://lilianweng.github.io/posts/2023-06-23-agent/"
TEXT = get_weburl_text(URL)
#TEXT

In [131]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(documents=chunks, embedding=cohere_embed())
print(vectorstore)

In [132]:
query = "What is the risks ?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)
print(len(docs))

–Consequences
 seeks to understand the 
potential consequences of risks, to create a network map of the global risk landscape. Respondents were provided 10 randomly selected global risks (from the full list of 34 global risks) and were then asked to select up to five global risks (from the full list) likely to be triggered by each of the 10 randomly selected risks. Results are illustrated in Figure 1.7.  In visual results, “Nodes: Risk influence” is based on a simple tally of all bidirectional relationships identified by respondents. “Edges: Relative influence” is based on a simple tally of the number of times the risk was identified as a consequence. However, visual results do not show all connections: weaker relationships identified by less than 25% of respondents were not included as edges.
 –Risk gover
nance asked respondents to identify
4


In [141]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 6})


In [134]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [143]:
SYS_PROMPT = """
You are a helpful, talkative and kind assistant for question-answering tasks, providing concise and low-level detailed answers from the context. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Respond kindly.
Question: {input}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", "{input}"),
    ]
)


In [149]:
# BUILD CHAIN 1
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain_0 = create_retrieval_chain(retriever, question_answer_chain)
#
response = rag_chain_0.invoke({"input": "can ou give me 6 main data in this report then, how to deal with them ?, also respond in french"})
response_stream = rag_chain_0.stream({"input": "What is Hallicination?"})
print("\n---------------- ANSWER ----------------\n")
print(response["answer"])
#for chunk in response_stream:
#    print(chunk, end="", flush=True)

print("\n\n---------------- SOURCES ----------------\n")
for document in response["context"]:
    print(document)
    print()


---------------- ANSWER ----------------

Bonjour ! I'd be happy to help you with that.

The main data from the report are:

1. Risk ranking: This table ranks countries based on their risk levels, with Kyrgyzstan being the highest-ranked country and Lesotho being the lowest.
2. Global adaptation score: This score measures a country's readiness to adapt to climate change, with a score ranging from 0 to 100.
3. Top 10 adaptation readiness: This table shows the top 10 countries with the highest adaptation readiness scores.
4. Share of respondents: This table shows the percentage of respondents who favor different approaches to addressing global risks, such as financial instruments, national regulations, and research and development.

Now, to deal with these data, I'd suggest the following:

1. Analyze the risk ranking: Identify countries with high risk levels and consider implementing strategies to mitigate these risks.
2. Focus on improving adaptation readiness: Countries with low adapt

In [102]:
# BUILD CHAIN 2
rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [103]:
rag_chain.invoke("What is Hallucination ?")

"Sure, I can explain that! Hallucination, in the context of the provided text, refers to a sequence of consecutive identical actions that lead to the same observation in an environment. It's like getting stuck in a loop, doing the same thing over and over again, and not making any progress or encountering something new. This definition is specifically relevant to the Reflexion framework mentioned in the text, where a heuristic function helps identify and stop such hallucinations. \n\nIs there anything else you'd like me to explain or provide more details on?"

In [81]:
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Sure, task decomposition is a technique used to break down a complex task into smaller, more manageable steps. It's like untangling a big knot by pulling on smaller threads! 

In the context of LLMs and autonomous agents, task decomposition is a way to improve the performance of the model on complex tasks. By instructing the model to think step by step, it can better plan and execute complicated tasks. This also helps interpret how the model is thinking and provide insights into its decision-making process. 

Hope that helps! Is there anything else you'd like to know or discuss further?